In [1]:
#conda install -c conda-forge geopy

In [19]:
# Importing required module 
from geopy.geocoders import Nominatim 
import pandas as pd

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [2]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



Import the data sets

In [3]:
# imports all zip codes of Tokyo
tokyo_zip = pd.read_csv("./data/tokyo_zip_latlong.csv", index_col = 0)
tokyo_zip.head()

,zip_code,都,区,町,Prefecture,District,Area,lat,long
0,1000000,東京都,千代田区,以下に掲載がない場合,TOKYO TO,CHIYODA KU,IKANIKEISAIGANAIBAAI,NaN,NaN
1,1020072,東京都,千代田区,飯田橋,TOKYO TO,CHIYODA KU,IIDABASHI,NaN,NaN
2,1020082,東京都,千代田区,一番町,TOKYO TO,CHIYODA KU,ICHIBANCHO,35.729056,139.378416
3,1010032,東京都,千代田区,岩本町,TOKYO TO,CHIYODA KU,IWAMOTOCHO,35.695600,139.775379
4,1010047,東京都,千代田区,内神田,TOKYO TO,CHIYODA KU,UCHIKANDA,35.691038,139.767290


In [4]:
tokyo_venues = pd.read_csv("./data/tokyo_venues.csv")
tokyo_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ICHIBANCHO,35.729056,139.378416,Kidoya (喜奴屋),35.730465,139.379067,Ramen Restaurant
1,ICHIBANCHO,35.729056,139.378416,7-Eleven (セブンイレブン 立川天王橋店),35.727325,139.382386,Convenience Store
2,ICHIBANCHO,35.729056,139.378416,7-Eleven (セブンイレブン 立川一番町4丁目店),35.731071,139.382957,Convenience Store
3,ICHIBANCHO,35.729056,139.378416,Lawson (ローソン 一番町四丁目店),35.731581,139.380197,Convenience Store
4,ICHIBANCHO,35.729056,139.378416,つかさ食堂,35.728742,139.377039,Sake Bar


In [7]:
# convert the venue data into one hot encoding
Tokyo_onehot = pd.get_dummies(tokyo_venues[['Venue Category']], prefix="", prefix_sep="")

# add area column back to dataframe
Tokyo_onehot["Area"] = tokyo_venues["Neighborhood"]
Tokyo_onehot.set_index("Area", inplace = True)
Tokyo_onehot.reset_index(inplace = True)

#grouping rows by area and by taking the mean of the frequency of occurrence of each category
Tokyo_grouped = Tokyo_onehot.groupby('Area').mean().reset_index()
Tokyo_grouped.head()

,Area,ATM,Acai House,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airfield,Airport,American Restaurant,...,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yakitori Restaurant,Yoga Studio,Yoshoku Restaurant,Yunnan Restaurant,Zoo,Zoo Exhibit
0,ABURADAI,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
1,ADACHI,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
2,AIHARAMACHI,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
3,AJIRO,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,AKABANE,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.0,0.0,0.0


In [30]:
# merge the data
zip_neighbor = tokyo_zip.merge(Tokyo_grouped, on = "Area")
zip_neighbor.dropna(inplace = True)

Plot geographic coordinates in a map

In [31]:
#obtain the location of Tokyo
address = 'Tokyo'

geolocator = Nominatim(user_agent="tk_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tokyo are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tokyo are 35.6828387, 139.7594549.


In [32]:
# create map of Tokyo using latitude and longitude values
map_tokyo = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map to visualize the neighborhoods
for lat, lng, label in zip(zip_neighbor['lat'], zip_neighbor['long'], zip_neighbor['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tokyo)  


map_tokyo